### Data Exploration and Cleaning

Run this notebook once to read the raw data file, transform it to a `.csv` file, conduct exploratory inspection and perform data cleaning. The data path(s) are loaded from the `.env` file. 

In [22]:
# Run once to load environment variables into the notebook environment
# If .env file is updated (rare), restart the kernel and rerun
%load_ext dotenv
%dotenv

# Install Excel engine dependency
%pip install openpyxl

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
Note: you may need to restart the kernel to use updated packages.


In [23]:
# Import relevant libraries
import os
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

# Read paths from .env, ensure "pre-processed" and "processed" directory exists
RAW_DATA_PATH = os.getenv("RAW_DATA_PATH", "./data/raw")
PRE_PROCESSED_DATA_PATH = os.getenv("PRE_PROCESSED_DATA_PATH", "./data/pre-processed")
PROCESSED_DATA_PATH = os.getenv("PROCESSED_DATA_PATH", "./data/processed")

raw_dir  = Path(RAW_DATA_PATH).expanduser().resolve()
pre_proc_dir = Path(PRE_PROCESSED_DATA_PATH).expanduser().resolve()
proc_dir = Path(PROCESSED_DATA_PATH).expanduser().resolve()
pre_proc_dir.mkdir(parents=True, exist_ok=True)
proc_dir.mkdir(parents=True, exist_ok=True)

# Load .xlsx file
xlsx_path = (raw_dir / os.getenv("RAW_DATA_FILE")).resolve()
assert xlsx_path.exists(), f"Expected: {xlsx_path} — run the fetch notebook or fix RAW_DATA_FILE"
assert xlsx_path.suffix.lower() == ".xlsx", f"Expected a .xlsx, got {xlsx_path.suffix}"

print("Using Excel:", xlsx_path)

Using Excel: C:\Users\jjsos\Documents\DSI_7\team_project\ds08_online-retail\data\raw\Online Retail.xlsx


In [24]:
# Read .xlsx file and write a pre-processing .csv

# Read "Online Retail" sheet from .xlsx
SHEET = os.getenv("RAW_DATA_SHEET") or 0 

# Set schemas (not cleaning)
dtypes = {
    "InvoiceNo": "string",
    "StockCode": "string",
    "Description": "string",
    "Quantity": "Int64",
    "UnitPrice": "float",
    "CustomerID": "string",
    "Country": "string",
}

# Read the .xlsx file once (Consider dd-mm-yyyy HH : MM date format for this dataset)
df_raw = pd.read_excel(
    xlsx_path,
    sheet_name=SHEET,
    dtype=dtypes,
    engine="openpyxl"
)

# Write a pre-processing .csv file
csv_path = pre_proc_dir / "online_retail_pre-processed.csv"
df_raw.to_csv(
    csv_path,
    index=False,
    encoding="utf-8-sig",
    na_rep=""
)

print(f"CSV saved to: {csv_path}")
print(f"Rows: {len(df_raw):,} | Columns: {len(df_raw.columns)}")
display(df_raw.head()) 

CSV saved to: C:\Users\jjsos\Documents\DSI_7\team_project\ds08_online-retail\data\pre-processed\online_retail_pre-processed.csv
Rows: 541,909 | Columns: 8


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [25]:
# Create dataframe (df)
online_retail_raw_df = pd.read_csv(csv_path)

# Overview inspection of df
online_retail_raw_df.info()

# 'CustomerID' and 'Description' has missing non-nulls.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [26]:
# Display head of the df for visual inspection of features and value distribution
online_retail_raw_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [27]:
# Inspect the country distribution per country to identify domestic vs international market ratio
inspection_country = (
    online_retail_raw_df["Country"]
      .value_counts(dropna=False)          # counts per country (include NaN if any)
      .rename("count")
      .to_frame()
      .assign(pct=lambda s: (s["count"] / s["count"].sum() * 100).round(2))
      .reset_index()
      .rename(columns={"index": "Country"})
)

inspection_country 
# The United Kingdom shares the sheer majority of the entry records at 91%.
# Consider standardizing country name lables.

,Country,count,pct
0,United Kingdom,495478,91.43
1,Germany,9495,1.75
2,France,8557,1.58
3,EIRE,8196,1.51
4,Spain,2533,0.47
5,Netherlands,2371,0.44
6,Belgium,2069,0.38
7,Switzerland,2002,0.37
8,Portugal,1519,0.28
9,Australia,1259,0.23


In [28]:
# Detecting missing values
online_retail_raw_df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [29]:
# Inspect CustomerID country of origin to determine further handling of missing values
missing_custuomerID = online_retail_raw_df["CustomerID"].isna()

# Fetch missing CustomerID by country
missing_by_country = (
    online_retail_raw_df.loc[missing_custuomerID]
    .groupby("Country", dropna=False)
    .size()
    .to_frame("missing_customer-id")
)

# Fetch total missing rows by country
total_by_country = (
    online_retail_raw_df
    .groupby("Country", dropna=False)
    .size()
    .to_frame("total_rows")
)

# Join and sort (counts only)
missing_by_country.join(total_by_country) \
    .fillna({"missing_customer-id-": 0}) \
    .astype({"missing_customer-id": "int64", "total_rows": "int64"}) \
    .sort_values("missing_customer-id", ascending=False)

# The United Kingdom shares the sheer majority of the missing CustomerID entries as well; consider dropping missing rows.
# Hong Kong's entries have CustomerID's missing, however it has a small amount of entries; consider dropping missing values.

,missing_customer-id,total_rows
Country,,
United Kingdom,133600,495478
EIRE,711,8196
Hong Kong,288,288
Unspecified,202,446
Switzerland,125,2002
France,66,8557
Israel,47,297
Portugal,39,1519
Bahrain,2,19


In [30]:
# Records/rows with missing CustomerID's will be excluded from the analysis, as they cannot be attributed to specific customers.
online_retail_purge01 = online_retail_raw_df.dropna(subset=['Description', 'CustomerID'])

online_retail_purge01.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


In [31]:
# Check for duplicates
online_retail_purge01.duplicated().sum()

np.int64(5225)

In [32]:
# Remove duplicates
online_retail_purge01.drop_duplicates(inplace=True)

online_retail_purge01.info()

<class 'pandas.core.frame.DataFrame'>
Index: 401604 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    401604 non-null  object 
 1   StockCode    401604 non-null  object 
 2   Description  401604 non-null  object 
 3   Quantity     401604 non-null  int64  
 4   InvoiceDate  401604 non-null  object 
 5   UnitPrice    401604 non-null  float64
 6   CustomerID   401604 non-null  float64
 7   Country      401604 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.6+ MB


C:\Users\jjsos\AppData\Local\Temp\ipykernel_5856\4276750689.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  online_retail_purge01.drop_duplicates(inplace=True)


In [33]:
# Look f there are negative and 0 values for Quantity and 0 values for UnitPrice
online_retail_purge01[["Quantity", "UnitPrice"]].describe()

,Quantity,UnitPrice
count,401604.000000,401604.000000
mean,12.183273,3.474064
std,250.283037,69.764035
min,-80995.000000,0.000000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,80995.000000,38970.000000


In [34]:
# Quantify negative and 0 values for Quantity and 0 values for UnitPrice
qty_neg = online_retail_purge01["Quantity"]  < 0
qty_zero = online_retail_purge01["Quantity"]  == 0
price_neg = online_retail_purge01["UnitPrice"] < 0
price_zero = online_retail_purge01["UnitPrice"] == 0

counts = pd.Series({
    "Quantity<0": int(qty_neg.sum()),
    "Quantity=0": int(qty_zero.sum()),
    "UnitPrice<0": int(price_neg.sum()),
    "UnitPrice=0": int(price_zero.sum()),
}).to_frame("count")
counts

,count
Quantity<0,8872
Quantity=0,0
UnitPrice<0,0
UnitPrice=0,40


In [35]:
# Remove 0 and negative values
zer_neg_drop = online_retail_purge01.index[(online_retail_purge01["Quantity"] < 0) | (online_retail_purge01["UnitPrice"] == 0)]
online_retail_purge02 = online_retail_purge01.drop(zer_neg_drop)

online_retail_purge02.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392692 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    392692 non-null  object 
 1   StockCode    392692 non-null  object 
 2   Description  392692 non-null  object 
 3   Quantity     392692 non-null  int64  
 4   InvoiceDate  392692 non-null  object 
 5   UnitPrice    392692 non-null  float64
 6   CustomerID   392692 non-null  float64
 7   Country      392692 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.0+ MB


In [36]:
# Change the data type for InvoiceDate
online_retail_purge02['InvoiceDate'] = pd.to_datetime(online_retail_purge02['InvoiceDate'])

# Change the data type for CustomerID from float to int
online_retail_purge02['CustomerID'] = online_retail_purge02['CustomerID'].astype('int64')

# Change the data type for other relevant columns to str
cols_str = ['InvoiceNo', 'StockCode', 'Description', 'Country']
online_retail_purge02[cols_str] = online_retail_purge02[cols_str].astype('string')

# Verify the datatypes
online_retail_purge02.dtypes

InvoiceNo      string[python]
StockCode      string[python]
Description    string[python]
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID              int64
Country        string[python]
dtype: object

In [37]:
# Standardize Country name labels
acronym_expander = {
    "EIRE": "Ireland",
    "USA": "United States",
    "RSA": "South Africa",
}

# Collapse extra spaces if any, expand acronyms, capitalize words if applicable
online_retail_purge02["Country"] = (
    online_retail_purge02["Country"]
        .astype("string")
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.upper()                           
        .replace(acronym_expander)           
        .str.title()                           
)

# Drop "Unspecified" and "European Community" labels
to_exclude = {"Unspecified", "European Community"}

online_retail_purge03 = (
    online_retail_purge02[~online_retail_purge02["Country"].isin(to_exclude)]
)

online_retail_purge03["Country"].value_counts()

Country
United Kingdom          349203
Germany                   9025
France                    8326
Ireland                   7226
Spain                     2479
Netherlands               2359
Belgium                   2031
Switzerland               1841
Portugal                  1453
Australia                 1181
Norway                    1071
Italy                      758
Channel Islands            747
Finland                    685
Cyprus                     603
Sweden                     450
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     245
Singapore                  222
Iceland                    182
United States              179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
South Africa                57
Lebanon                     45
Lithuania                   35
Brazil                      32


In [38]:
# Final inspection of cleaned dataset prior to creating clean .csv
online_retail_purge03.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392391 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392391 non-null  string        
 1   StockCode    392391 non-null  string        
 2   Description  392391 non-null  string        
 3   Quantity     392391 non-null  int64         
 4   InvoiceDate  392391 non-null  datetime64[ns]
 5   UnitPrice    392391 non-null  float64       
 6   CustomerID   392391 non-null  int64         
 7   Country      392391 non-null  string        
dtypes: datetime64[ns](1), float64(1), int64(2), string(4)
memory usage: 26.9 MB


In [39]:
# Create clean, processed dataset .csv
out_csv = proc_dir / "online_retail_processed.csv"
online_retail_purge03.to_csv(out_csv, index=False, encoding="utf-8-sig")